# Refinements

Possible improvements to `fastf1` package.

## Improve ergast API

Add additional ergast API requests:

In [51]:
from fastf1.ergast import base_url, _parse_ergast, _parse_json_response, _headers
from fastf1.api import Cache
from pandas import json_normalize

In [59]:
def _parse_ergast(data, table="RaceTable:Races", as_df=False):
    """Parse specified table from ergast data."""
    data_table = data['MRData']
    for t in table.split(":"):
        if t:
            data_table = data_table[t]
    return data_table if not as_df else json_normalize(data_table)

In [53]:
#http://ergast.com/api/f1/drivers
#http://ergast.com/api/f1/2010/drivers
#http://ergast.com/api/f1/2010/2/drivers
def fetch_driver(year, gp=None, as_df=False):
    """Get driver information from ergast API by year and optionally by race."""
    if gp is None:
        url = f"{base_url}/{year}/drivers.json"
    else:
        url = f"{base_url}/{year}/{gp}/drivers.json"
    return _parse_ergast(_parse_json_response(
        Cache.requests_get(url, headers=_headers)),
                         "DriverTable:Drivers", as_df
    )

In [55]:
fetch_driver(2022, as_df=True).head()

,driverId,permanentNumber,code,url,givenName,familyName,dateOfBirth,nationality
0,albon,23,ALB,http://en.wikipedia.org/wiki/Alexander_Albon,Alexander,Albon,1996-03-23,Thai
1,alonso,14,ALO,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando,Alonso,1981-07-29,Spanish
2,bottas,77,BOT,http://en.wikipedia.org/wiki/Valtteri_Bottas,Valtteri,Bottas,1989-08-28,Finnish
3,gasly,10,GAS,http://en.wikipedia.org/wiki/Pierre_Gasly,Pierre,Gasly,1996-02-07,French
4,hamilton,44,HAM,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis,Hamilton,1985-01-07,British


In [42]:
from unidecode import unidecode

def driver_translate(year, gp=None):
    """Get a driver translate lookup dict via ergast API."""
    def _n(d):
        #Normalise driver name
        return unidecode(f'{d["givenName"]} {d["familyName"]}').lower()

    drivers = fetch_driver(year, gp)
    _DRIVER_TRANSLATE = {d["code"]: _n(d) for d in drivers}
    return _DRIVER_TRANSLATE

In [160]:
def ergast_driver_code_number(year, gp=None, typ="code", index="driverId"):
    """Get driver codes via ergast API."""
    _driver_details = fetch_driver(year, gp, as_df=True)
    _driver_details["permanentNumber"] = _driver_details["permanentNumber"].astype(int)
    # The typ value sets the lookup value against the ergast driverId.
    # By default, the driverId provides keys fro the returned dict.
    # The typ can also be autoset via the index value
    # if we want to use the code or number as the key, and driverId as the value.
    typ = index if index and index not in [typ, "driverId"] else typ
    typ = "permanentNumber" if typ.startswith("num") else typ
    index = "permanentNumber" if index.startswith("num") else index
    # Ideally, return a dictionary
    if typ in ["code", "permanentNumber"]:
        return _driver_details[["driverId", typ]].set_index(index).squeeze().to_dict()
    # Otherwise, provide a lookup dataframe
    return _driver_details[["code", "permanentNumber", "driverId"]]
    

Automate creation of `DRIVER_TRANSLATE` lookup in `fastf1.plotting`:

In [161]:
# Manually defined lookup
DRIVER_TRANSLATE = {'LEC': 'charles leclerc', 'SAI': 'carlos sainz',
                    'VER': 'max verstappen', 'PER': 'sergio perez',
                    'RIC': 'daniel ricciardo', 'NOR': 'lando norris',
                    'ALO': 'fernando alonso', 'OCO': 'esteban ocon',
                    'BOT': 'valtteri bottas', 'ZHO': 'zhou guanyu',
                    'GAS': 'pierre gasly', 'TSU': 'yuki tsunoda',
                    'MAG': 'kevin magnussen', 'MSC': 'mick schumacher',
                    'VET': 'sebastian vettel', 'HUL': 'nico hulkenberg',
                    'STR': 'lance stroll',
                    'HAM': 'lewis hamilton', 'RUS': 'george russell',
                    'ALB': 'alexander albon', 'LAT': 'nicholas latifi'}

Create a lookup from ergast identifiers to driver codes:

In [162]:
ERGAST_CODES = ergast_driver_code_number(2022)
ERGAST_CODES

{'albon': 'ALB',
 'alonso': 'ALO',
 'bottas': 'BOT',
 'gasly': 'GAS',
 'hamilton': 'HAM',
 'hulkenberg': 'HUL',
 'latifi': 'LAT',
 'leclerc': 'LEC',
 'kevin_magnussen': 'MAG',
 'norris': 'NOR',
 'ocon': 'OCO',
 'perez': 'PER',
 'ricciardo': 'RIC',
 'russell': 'RUS',
 'sainz': 'SAI',
 'mick_schumacher': 'MSC',
 'stroll': 'STR',
 'tsunoda': 'TSU',
 'max_verstappen': 'VER',
 'vettel': 'VET',
 'zhou': 'ZHO'}

We can also get lap times:

In [163]:
import pandas as pd

#http://ergast.com/api/f1/2011/5/laps/1
#http://ergast.com/api/f1/2011/5/drivers/alonso/laps/1
def fetch_laptimes(year, gp, lap=None, driver=None, as_df=False):
    """Get laptime information from ergast API by year and race,
        and optionally by driver and/or lap."""
    url = f"{base_url}/{year}/{gp}"
    if driver:
        url = f"{url}/drivers/{driver}"
    url = f"{url}/laps"
    if lap:
        url = f"{url}/{lap}"
    url = f"{url}.json"
    _laps = _parse_ergast(_parse_json_response(
        Cache.requests_get(url, headers=_headers))
    )[0]["Laps"]
    if as_df:
        _df = pd.DataFrame(_laps[0]["Timings"])
        _df['driverId'] = _df['driverId'].map(ERGAST_CODES)
        return _df
    return _laps

In [165]:
fetch_laptimes(2022, 2, 1)

[{'number': '1',
  'Timings': [{'driverId': 'perez', 'position': '1', 'time': '1:35.991'},
   {'driverId': 'leclerc', 'position': '2', 'time': '1:37.088'},
   {'driverId': 'max_verstappen', 'position': '3', 'time': '1:38.082'},
   {'driverId': 'sainz', 'position': '4', 'time': '1:38.635'},
   {'driverId': 'ocon', 'position': '5', 'time': '1:39.900'},
   {'driverId': 'russell', 'position': '6', 'time': '1:40.421'},
   {'driverId': 'alonso', 'position': '7', 'time': '1:41.458'},
   {'driverId': 'bottas', 'position': '8', 'time': '1:42.123'},
   {'driverId': 'kevin_magnussen', 'position': '9', 'time': '1:42.705'},
   {'driverId': 'gasly', 'position': '10', 'time': '1:43.106'},
   {'driverId': 'norris', 'position': '11', 'time': '1:43.333'},
   {'driverId': 'stroll', 'position': '12', 'time': '1:43.828'},
   {'driverId': 'ricciardo', 'position': '13', 'time': '1:44.274'},
   {'driverId': 'hamilton', 'position': '14', 'time': '1:44.788'},
   {'driverId': 'albon', 'position': '15', 'time': '